## Data Wrangling of Instacart Dataset

### Introduction

Curious abut the food Americans ear ? Instacard  release the first public dataset "The Instacart Online Grocery Shopping Dataset 2017". This anonymized dataset contains a sample of over 3 million grocery orders from more than 200,000 Instacart users.

For each user, Instacart provides between 4 to 100 of their orders, with teh sequence of products purchsed in each order. Week and hour of day the order was placed, and a relative measure of time between orders. 

**DATA COLUMNS**

_**orders** 

    order_id: order identifier
    user_id: customer identifier
    eval_set: which evaluation set this order belongs in (see SET described below)
    order_number: the order sequence number for this user (1 = first, n = nth)
    order_dow: the day of the week the order was placed on
    order_hour_of_day: the hour of the day the order was placed on
    days_since_prior: days since the last order, capped at 30 (with NAs for order_number = 1)

_**products**

    product_id: product identifier
    product_name: name of the product
    aisle_id: foreign key
    department_id: foreign key

_**aisles**

    aisle_id: aisle identifier
    aisle: the name of the aisle

_**departments**

    department_id: department identifier
    department: the name of the department

_**order_products__SET**

    order_id: foreign key
    product_id: foreign key
    add_to_cart_order: order in which each product was added to cart
    reordered: 1 if this product has been ordered by this user in the past, 0 otherwise

where SET is one of the four following evaluation sets (eval_set in orders):

    "prior": orders prior to that users most recent order 
    "train": training data supplied to participants 
    "test": test data reserved for machine learning competitions 



### Business Problem

The objective is to predict which products will be in a user's next order. The dataset is anonymized and contains a sample of over 3 million gracoery orders from more than 200,000 Instacart users. For each user, instacard provided between 4 and 1900 of their orders, with the order of products which were added to card

### Read in data

_**products** :  data set has 49688 different products

_**aisle** :  data set has 134 different aisle details

_**departments**: data set contains 21 different departmetn details

_**order**: Order datset holds 3 million differnet orders for 200k plus customers 

_**product_ordered** : data set hold 30 million plus order details 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import warnings

warnings.filterwarnings('ignore')

In [8]:
products  = pd.read_csv('../data/products.csv')
aisles = pd.read_csv('../data/aisles.csv')
departments = pd.read_csv('../data/departments.csv')
orders = pd.read_csv('../data/orders.csv')
prior_order = pd.read_csv('../data/order_products__prior.csv')
train_order =  pd.read_csv('../data/order_products__train.csv')


In [9]:
print('aisles shape :', aisles.shape)
print('departments shape :',departments.shape)
print('prior_order shape :', prior_order.shape)
print('train_order shape :', train_order.shape)
print('orders shape :', orders.shape)
print('products shape :', products.shape)

aisles shape : (134, 2)
departments shape : (21, 2)
prior_order shape : (32434489, 4)
train_order shape : (1384617, 4)
orders shape : (3421083, 7)
products shape : (49688, 4)


### A Look into Initial Dataset

In [172]:
print(products.shape)
products.nunique()

(49688, 4)


product_id       49688
product_name     49688
aisle_id           134
department_id       21
dtype: int64

In [173]:
products.head(4)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1


In [13]:
aisle.nunique()

aisle_id    134
aisle       134
dtype: int64

In [49]:
print(departments.nunique())
print(departments.head(4))

department_id    21
department       21
dtype: int64
   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce


In [20]:
print(orders.nunique())
print(orders.head(4))


order_id                  3421083
user_id                    206209
eval_set                        3
order_number                  100
order_dow                       7
order_hour_of_day              24
days_since_prior_order         31
dtype: int64
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  


In [39]:
print(products_ordered.nunique())
print(products_ordered.shape)



order_id             3214874
product_id             49677
add_to_cart_order        145
reordered                  2
dtype: int64
(32434489, 4)


In [10]:
#check if null values are present only for 1st orders of all users
orders[orders['order_number']==1].isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

__Conclusion__:

For every user's first order , days_since_prior_order is left empty .
It is safe to impute 0 here .


In [181]:
train_dataset.nunique()

order_id             131209
product_id            39123
add_to_cart_order        80
reordered                 2
dtype: int64

### Merging Data sets

In [40]:
prior_train_orders  = pd.concat([prior_order, train_order]).sort_values(by=['order_id'])
print("Shape of prior_train_orders :", prior_train_orders.shape)
prior_train_orders.head(20)

Shape of prior_train_orders : (33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
1,1,11109,2,1
0,2,33120,1,1
1,2,28985,2,1


In [41]:
#merge prior_train_orders and products

prior_train_orders = pd.merge(left = prior_train_orders, right = products,
                             left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)

print("Shape of prior_train_orders :", prior_train_orders.shape)
prior_train_orders.head(3)

Shape of prior_train_orders : (33819106, 7)


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
1,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15
2,1,13176,6,0,Bag of Organic Bananas,24,4


In [42]:
#merge prior_train_orders and aisles

prior_train_orders = pd.merge(left = prior_train_orders, right = aisles,\
                             left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)

print("Shape of prior_train_orders :", prior_train_orders.shape)

Shape of prior_train_orders : (33819106, 8)


In [43]:
#merge prior_train_orders and departments

prior_train_orders = pd.merge(left = prior_train_orders, right = departments,
                             left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)

print("Shape of prior_train_orders :", prior_train_orders.shape)
prior_train_orders.head(3)

Shape of prior_train_orders : (33819106, 9)


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce
2,1,13176,6,0,Bag of Organic Bananas,24,4,fresh fruits,produce


### Order hour of day column

this column is changed to four differnet category for analysis

In [23]:
def dayrange(dataset):
    dataset['order_hour_of_day'] = dataset['order_hour_of_day'].astype(int)
    dayrange =[]
    for i,val in enumerate(dataset['order_hour_of_day']):
        if val >= 0 and val <= 6:
           dayrange.append('Morning')
        elif val > 6 and  val <= 12:
           dayrange.append('Afternoon')
        elif val > 12 and val <= 18:
           dayrange.append('Evening')
        else:
           dayrange.append('Night')
        
            
    dataset['dayrange']  = dayrange
    return dataset

                           
dayrange(orders)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,dayrange
0,2539329,1,prior,1,2,8,NaN,Afternoon
1,2398795,1,prior,2,3,7,15.0,Afternoon
2,473747,1,prior,3,3,12,21.0,Afternoon
3,2254736,1,prior,4,4,7,29.0,Afternoon
4,431534,1,prior,5,4,15,28.0,Evening
...,...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0,Evening
3421079,1854736,206209,prior,11,4,10,30.0,Afternoon
3421080,626363,206209,prior,12,1,12,18.0,Afternoon
3421081,2977660,206209,prior,13,1,12,7.0,Afternoon


### order_dow column 

with 0 as Sunday, each orders dow is convered to weekdays

In [24]:
weekday ={ 1: "Monday",
           2: "Tuesday",
           3: "Wednesday",
           4: "Thursday",
           5: "Friday",
           6: "Saturday",
           0: "Sunday"}

orders['order_day'] = [ weekday[d] for d in orders['order_dow']]

In [25]:
orders['order_day'] = orders['order_day'].astype('category')
orders['dayrange'] = orders['dayrange'].astype('category')

In [26]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 9 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   eval_set                object  
 3   order_number            int64   
 4   order_dow               int64   
 5   order_hour_of_day       int32   
 6   days_since_prior_order  float64 
 7   dayrange                category
 8   order_day               category
dtypes: category(2), float64(1), int32(1), int64(4), object(1)
memory usage: 176.2+ MB


In [27]:
orders.eval_set.unique()

array(['prior', 'train', 'test'], dtype=object)

In [31]:
max_purchase = orders.groupby('user_id')['order_number'].max().reset_index(name='max_purchasecount')

#check whether to use inner or left
orders = orders.merge(max_purchase, on ='user_id',how ='inner')


In [32]:
print(' Number of unique orders ==> {}'.format(orders.order_id.nunique()))
print(' Number of Customers ==> {}'.format(orders.user_id.nunique()))
print(' Maximum order placed by Customer ==> {}'.format(orders['max_purchasecount'].max()))
print(' Minimum order placed by Customer ==> {}'.format(orders['max_purchasecount'].min()))

 Number of unique orders ==> 3421083
 Number of Customers ==> 206209
 Maximum order placed by Customer ==> 100
 Minimum order placed by Customer ==> 4


In [35]:
def missing(dataset):
    columns = dataset.columns
    print('MISSING ROWS per COLUMN')
    for column in columns:
        percentage = (dataset[column].isnull().sum() / len(dataset)) * 100
        print('{}: {}, {:0.2f}%'.format(column, dataset[column].isnull().sum(), percentage))
        
missing(orders)

MISSING ROWS per COLUMN
order_id: 0, 0.00%
user_id: 0, 0.00%
eval_set: 0, 0.00%
order_number: 0, 0.00%
order_dow: 0, 0.00%
order_hour_of_day: 0, 0.00%
days_since_prior_order: 206209, 6.03%
dayrange: 0, 0.00%
order_day: 0, 0.00%
max_purchasecount: 0, 0.00%


In [36]:
orders = pd.get_dummies(orders, columns=['order_day','dayrange'],prefix='Col')

In [37]:
orders.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,max_purchasecount,Col_Friday,Col_Monday,Col_Saturday,Col_Sunday,Col_Thursday,Col_Tuesday,Col_Wednesday,Col_Afternoon,Col_Evening,Col_Morning,Col_Night
0,2539329,1,prior,1,2,8,NaN,11,False,False,False,False,False,True,False,True,False,False,False
1,2398795,1,prior,2,3,7,15.0,11,False,False,False,False,False,False,True,True,False,False,False
2,473747,1,prior,3,3,12,21.0,11,False,False,False,False,False,False,True,True,False,False,False
3,2254736,1,prior,4,4,7,29.0,11,False,False,False,False,True,False,False,True,False,False,False
4,431534,1,prior,5,4,15,28.0,11,False,False,False,False,True,False,False,False,True,False,False
5,3367565,1,prior,6,2,7,19.0,11,False,False,False,False,False,True,False,True,False,False,False
6,550135,1,prior,7,1,9,20.0,11,False,True,False,False,False,False,False,True,False,False,False
7,3108588,1,prior,8,1,14,14.0,11,False,True,False,False,False,False,False,False,True,False,False
8,2295261,1,prior,9,1,16,0.0,11,False,True,False,False,False,False,False,False,True,False,False
9,2550362,1,prior,10,4,8,30.0,11,False,False,False,False,True,False,False,True,False,False,False


### Let's analyse product and order dataset

**Banana** seems to be  most ordered as well as reordered item. From both most ordered as well as re-ordered products, fresh produce section hold top spot

In [44]:
#merge prior_train_orders with orders
# since orders have all prior/train/test data, we will merge only prior and train orders for EDA , but leave test orders untouched

prior_train_orders = pd.merge(left = prior_train_orders, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)

print("Shape :", prior_train_orders.shape)
prior_train_orders.head(10)

Shape : (33819106, 27)


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,...,Col_Monday,Col_Saturday,Col_Sunday,Col_Thursday,Col_Tuesday,Col_Wednesday,Col_Afternoon,Col_Evening,Col_Morning,Col_Night
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,112108,...,False,False,False,True,False,False,True,False,False,False
1,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,112108,...,False,False,False,True,False,False,True,False,False,False
2,1,13176,6,0,Bag of Organic Bananas,24,4,fresh fruits,produce,112108,...,False,False,False,True,False,False,True,False,False,False
3,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,112108,...,False,False,False,True,False,False,True,False,False,False
4,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,112108,...,False,False,False,True,False,False,True,False,False,False
5,1,47209,7,0,Organic Hass Avocado,24,4,fresh fruits,produce,112108,...,False,False,False,True,False,False,True,False,False,False
6,1,22035,8,1,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs,112108,...,False,False,False,True,False,False,True,False,False,False
7,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,112108,...,False,False,False,True,False,False,True,False,False,False
8,2,17794,6,1,Carrots,83,4,fresh vegetables,produce,202279,...,False,False,False,False,False,False,True,False,False,False
9,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry,202279,...,False,False,False,False,False,False,True,False,False,False


In [46]:
product_count = prior_train_orders.groupby('product_name')['order_id'].count().reset_index(name='count').sort_values(by ='count',ascending=False)
print("Most ordered products \n")
print(product_count.head(10))

Most ordered products 

                 product_name   count
3677                   Banana  491291
3472   Bag of Organic Bananas  394930
31923    Organic Strawberries  275577
28843    Organic Baby Spinach  251705
30300    Organic Hass Avocado  220877
28807         Organic Avocado  184224
22415             Large Lemon  160792
42908            Strawberries  149445
23422                   Limes  146660
32481      Organic Whole Milk  142813


In [47]:
product_count = prior_train_orders[prior_train_orders['reordered'] == 1].groupby('product_name')['reordered'].count().reset_index(name='count').sort_values(by ='count',ascending=False)
print("Most Re-ordered products \n")
print(product_count.head(10))

Most Re-ordered products 

                 product_name   count
3285                   Banana  415166
3095   Bag of Organic Bananas  329275
29270    Organic Strawberries  214448
26334    Organic Baby Spinach  194939
27718    Organic Hass Avocado  176173
26299         Organic Avocado  140270
29792      Organic Whole Milk  118684
20443             Large Lemon  112178
28741     Organic Raspberries  109688
39385            Strawberries  104588


In [36]:
df_productdetails.shape

(33819106, 7)

### Drop Rows with Missing Data



In [48]:
missing(prior_train_orders)

MISSING ROWS per COLUMN
order_id: 0, 0.00%
product_id: 0, 0.00%
add_to_cart_order: 0, 0.00%
reordered: 0, 0.00%
product_name: 0, 0.00%
aisle_id: 0, 0.00%
department_id: 0, 0.00%
aisle: 0, 0.00%
department: 0, 0.00%
user_id: 0, 0.00%
eval_set: 0, 0.00%
order_number: 0, 0.00%
order_dow: 0, 0.00%
order_hour_of_day: 0, 0.00%
days_since_prior_order: 2078068, 6.14%
max_purchasecount: 0, 0.00%
Col_Friday: 0, 0.00%
Col_Monday: 0, 0.00%
Col_Saturday: 0, 0.00%
Col_Sunday: 0, 0.00%
Col_Thursday: 0, 0.00%
Col_Tuesday: 0, 0.00%
Col_Wednesday: 0, 0.00%
Col_Afternoon: 0, 0.00%
Col_Evening: 0, 0.00%
Col_Morning: 0, 0.00%
Col_Night: 0, 0.00%


In [29]:
df_orders = df_productdetails.merge(orders, how ='left', on='order_id').sort_values('order_id')
df_orders.head(10)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,...,Col_Monday,Col_Saturday,Col_Sunday,Col_Thursday,Col_Tuesday,Col_Wednesday,Col_Afternoon,Col_Evening,Col_Morning,Col_Night
0,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,112108,train,4,...,False,False,False,True,False,False,True,False,False,False
1,1,47209,7,0,Organic Hass Avocado,24,4,112108,train,4,...,False,False,False,True,False,False,True,False,False,False
2,1,10246,3,0,Organic Celery Hearts,83,4,112108,train,4,...,False,False,False,True,False,False,True,False,False,False
3,1,49302,1,1,Bulgarian Yogurt,120,16,112108,train,4,...,False,False,False,True,False,False,True,False,False,False
4,1,13176,6,0,Bag of Organic Bananas,24,4,112108,train,4,...,False,False,False,True,False,False,True,False,False,False
5,1,22035,8,1,Organic Whole String Cheese,21,16,112108,train,4,...,False,False,False,True,False,False,True,False,False,False
6,1,49683,4,0,Cucumber Kirby,83,4,112108,train,4,...,False,False,False,True,False,False,True,False,False,False
7,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,112108,train,4,...,False,False,False,True,False,False,True,False,False,False
16,2,33120,1,1,Organic Egg Whites,86,16,202279,prior,3,...,False,False,False,False,False,False,True,False,False,False
15,2,40141,7,1,Original Unflavored Gelatine Mix,105,13,202279,prior,3,...,False,False,False,False,False,False,True,False,False,False


In [49]:
prior_train_orders.rename(columns = {'Col_Monday': 'Monday',
                    'Col_Tuesday' : 'Tuesday',
                    'Col_Wednesday': 'Wednesday',
                      'Col_Thursday': 'Thursday',
                      'Col_Friday': 'Friday',
                      'Col_Saturday': 'Saturday',
                      'Col_Sunday': 'Sunday' ,
                      'Col_Morning': 'Morning',
                      'Col_Night' : 'Night',
                      'Col_Afternoon' : 'Afternoon', 
                      'Col_Evening'   : 'Evening' },inplace = True)

In [43]:
df_orders.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered',
       'product_name', 'aisle_id', 'department_id', 'user_id', 'eval_set',
       'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'max_purchasecount', 'Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'Afternoon',
       'Evening', 'Morning', 'Night'],
      dtype='object')

In [51]:
prior_train_orders.shape

(33819106, 27)

## Output modified Dataset to New CSV File

In [53]:
prior_train_orders.to_csv('../Data/ipr.csv',index = False)

In [33]:
print(innerdt.shape)

(0, 7)


In [54]:
del prior_train_orders